# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-cou

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
#giving one example is the equivalent of one shot prompting 

link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [17]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
Include links to About page, Company page, Careers/Jobs pages if available,
Err on the side of including too many links, rather than to few
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [7]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
htt

In [8]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        #constrain the model to infererence to JSON only
        response_format={"type": "json_object"}
    )

    result = response.choices[0].message.content
    links = json.loads(result)
    return links






In [12]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'blog', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'portfolio page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'portfolio page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'article',
   'url': 'https://edwarddonner.com/2025/11/11/ai-live-event/'},
  {'type': 'article',
   'url': 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/'},
  {'type': 'article',
   'url': 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/'},
  {'type': 'article',
   'url': 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type

In [10]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [11]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'blog', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'linkedin profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/11/11/ai-live-event/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/'},
  {'type': 'twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook page',
   'url': 'https://www.facebook.com/edward.donner.5

In [13]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [18]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 6 relevant links


{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [19]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 8 relevant links


{'links': [{'type': 'company page', 'url': 'https://huggingface.co/'},
  {'type': 'about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'code repository', 'url': 'https://github.com/huggingface'},
  {'type': 'social media page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'social media page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'status page', 'url': 'https://status.huggingface.co/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [20]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [21]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
tencent/HunyuanVideo-1.5
Updated
2 days ago
•
1.68k
•
685
facebook/sam3
Updated
5 days ago
•
115k
•
672
black-forest-labs/FLUX.2-dev
Updated
about 9 hours ago
•
59
•
420
Supertone/supertonic
Updated
4 days ago
•
4.11k
•
280
tencent/HunyuanOCR
Updated
17 minutes ago
•
626
•
212
Browse 1M+ models
Spaces
Running
on
Zero
MCP
Featured
1.28k
Qwen Image Edit Camera Control
🎬
1.28k
Fast 4 step inference with Qwen Image Edit 2509
Running
on
CPU Upgrade
Featured
2.43k
The Smol Training Playbook
📚
2.43k
The secrets to building world-class LLMs


In [33]:
# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""


In [23]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [24]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 19 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ntencent/HunyuanVideo-1.5\nUpdated\n2 days ago\n•\n1.68k\n•\n685\nfacebook/sam3\nUpdated\n5 days ago\n•\n115k\n•\n672\nblack-forest-labs/FLUX.2-dev\nUpdated\nabout 9 hours ago\n•\n59\n•\n420\nSupertone/supertonic\nUpdated\n4 days ago\n•\n4.11k\n•\n281\ntencent/HunyuanOCR\nUpdated\n19 minutes ago\n•\n626\n•\n212\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\nMCP\nFeatured\n1.28k\nQwen Image E

In [28]:
def create_brochure(company_name,url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],)
    result = response.choices[0].message.content
    display(Markdown(result))


In [29]:
# def create_brochure(company_name, url):
#     response = openai.chat.completions.create(
#         model="gpt-4.1-mini",
#         messages=[
#             {"role": "system", "content": brochure_system_prompt},
#             {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
#         ],
#     )
#     result = response.choices[0].message.content
#     display(Markdown(result))

In [30]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 5 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is a pioneering AI community dedicated to building the future of machine learning. It serves as the premier platform where a global machine learning community collaborates seamlessly on models, datasets, and AI applications. Hugging Face empowers developers, researchers, and enterprises to create, discover, and innovate the next generation of AI technologies.

---

## What We Offer

- **Models:** Access and contribute to over **1 million machine learning models** spanning diverse AI modalities such as text, image, video, audio, and even 3D.
- **Datasets:** Explore and share over **250,000 datasets** that fuel AI research and development.
- **Spaces:** Deploy and engage with interactive AI applications with zero setup, powered by community and enterprise contributions.
- **Community:** Join a vibrant, open community collaborating on shaping the future of AI.
- **Docs & Learning:** Comprehensive documentation to help users at all levels create and innovate faster.

---

## Key Features

- **Collaboration Platform:** Host and collaborate on unlimited public models, datasets, and applications in a shared open environment.
- **Open Source Stack:** Move faster by leveraging Hugging Face’s robust and trusted open source tools.
- **Multi-Modality Support:** Build AI solutions across multiple data types – from natural language to 3D.
- **Portfolio Building:** Showcase your machine learning projects and build a public professional portfolio.
- **Compute and Enterprise Solutions:** Paid compute power and enterprise services to accelerate AI workloads with enterprise-grade security and dedicated support.

---

## Serving a Global Customer Base

Hugging Face supports a wide range of users including academia, startups, large enterprises, and AI enthusiasts. Top organizations utilize Hugging Face for leveraging cutting-edge models in fields like autonomous vehicles, natural language processing, computer vision, and more.

Enterprise customers benefit from:

- Advanced AI platform capabilities tailored for organizational scale
- Single Sign-On (SSO) for secure identity management
- Regional data management and compliance
- Detailed audit logs and resource control for governance

---

## Company Culture

Hugging Face fosters an inclusive and vibrant community culture emphasizing:

- Open collaboration and knowledge sharing  
- Innovation driven by collective intelligence  
- Transparency and openness in machine learning development  
- Empowering members to learn, build, and contribute freely  
- Supporting a welcoming environment for AI professionals worldwide

---

## Careers at Hugging Face

Join a fast-growing company at the forefront of AI innovation. Hugging Face seeks talented individuals passionate about machine learning, open-source development, and community-building.

**Current opportunities:** Machine Learning Engineers, software developers, research scientists, community managers, and enterprise solutions experts.

Working at Hugging Face means collaborating with some of the best minds in AI, enjoying a culture that values experimentation, empowerment, and continuous learning.

---

## Get Started

Explore the wealth of resources and AI apps Hugging Face offers:

- Browse over **1M+ models** and **400k+ applications**
- Explore diverse datasets and build your AI portfolio
- Sign up to participate, share, and accelerate your AI journey: [huggingface.co](https://huggingface.co)

For enterprises, explore tailored plans starting at $20/user/month or contact sales for bespoke solutions.

---

**Hugging Face — The AI community building the future.**  
Join us to create, discover, and collaborate on machine learning better, faster, and smarter.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
          
        stream=True
    )    
    response = ""
    #set basic markdown (blank) - this is then updated as chunks arrive
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        #append new content - based on the delta - the additional content only
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [34]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 16 relevant links


# Welcome to Hugging Face: The Playground of AI Wizards 🤖✨

---

## Who Are We?  
Think of us as the social network for machine learning — but instead of cat memes, we host **over 1 million AI models, 250k datasets, and 400k applications**. Yes, it’s basically the Netflix of AI, but for builders, tinkerers, and dreamers shaping the future.

Our mantra? **Build AI together, build AI better**. We’re the friendly neighborhood hub where the global machine learning community shares, experiments, and collaborates to invent the next big thing in AI. Because great minds don’t just think alike — they share code.

---

## What Goes On Here?  

- **Models Galore:** From Tencent’s latest video generation marvels to Facebook’s state-of-the-art segmentation models, we’ve got AI models for every flavor and fancy. Feel like creating videos from images? We got that. Fancy automating OCR or generating 3D depth maps? We did that yesterday.

- **Diving into Datasets:** With **250,000+ datasets**, including autonomous vehicle data from Nvidia and multilingual speech corpora from Facebook, your AI appetite for data will never go hungry.

- **Spaces for Creativity:** Running your own AI demos or apps? Welcome to *Spaces* — a place to deploy and showcase your AI experiments with the world. It’s like AI’s version of an art gallery, but with more robots and fewer berets.

- **Open Source Superpowers:** We ride the open-source wave, providing the tools and infrastructure so you can move from idea to impact faster than you can say “gradient descent.”

---

## Our Culture: Nerdy, Friendly, and a Bit Magical 🧙‍♂️💻  

- **Collaboration is King:** We believe AI is best built with many hands on deck and many minds in sync. Our community is our secret sauce — from newbies learning the ropes to pros pushing the state of the art.

- **Ethics First:** Building AI is cool. Building *ethical* AI? That’s the future. We’re committed to fostering an open, responsible, and inclusive AI ecosystem.

- **Diversity of Modalities:** Text, images, audio, video, 3D — if data can be modeled, we welcome it with open arms and humming GPUs.

---

## Who Uses Hugging Face?  

- **Machine Learning Engineers and Scientists:** To build, experiment, and share cutting-edge models.
- **AI Researchers:** For collaborating on grand challenges and pushing the boundaries of what machines can learn.
- **Enterprises:** Leveraging our platform and paid compute to accelerate AI adoption and innovation, all while customizing solutions at scale.
- **Students and Hobbyists:** To learn, build portfolios, and showcase their AI superpowers to the world.

---

## Careers – Join the AI Dream Team! 🚀  

At Hugging Face, you won’t just write code — you’ll be part of an AI revolution. We look for passionate innovators who thrive in a community-driven, open-source culture. Whether you’re into building open source libraries, running AI infrastructure, or creating AI-powered experiences, your playground awaits.

*Perks?* Infinite curiosity encouraged, coffee (and codes) on tap, limitless access to the freshest AI models, and a chance to shape the future — one model at a time.

---

## Ready to Get Hugged by the Future?  

- Build your AI portfolio  
- Collaborate with the sharpest minds  
- Browse 1 million+ models and 250k datasets  
- Deploy your own AI spaces and applications  
- Join a community that’s not just creating AI — we’re creating *the* future of AI.

**Dive in at [huggingface.co](https://huggingface.co)**  
Because the future doesn’t build itself. It codes, collaborates, and creates — together.

---

*Hugging Face: Where AI meets Hugging.*  
*(We promise it’s friendlier than it sounds.)*

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>